In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import json
import pandas as pd
import numpy as np
#!pip install pandas_datareader to validate crypto pricing with specific date
import pandas_datareader as web
import datetime as dt
from datetime import date
import seaborn as sns
from numerize import numerize
import torch #torch first before matplotlib, otherwise the library will crash the environment. 
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import string
#NLP libraries
import re
# Import nltk modules and download dataset
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download("omw-1.4")

stop = set(stopwords.words('english'))

#import FinBert library
from textblob import TextBlob
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\eikde\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
#extract data libraries
import requests
import dateutil.parser
import unicodedata
import time
from searchtweets import load_credentials

In [4]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

print(os.getcwd())

C:\Users\eikde\source\repos\UM_Crypto_Sentiment_Prediction


In [5]:
from pprint import pprint
from transformers import AutoModelForSequenceClassification

from finbert import *
import finbert.utils as tools
from finbert.finbert import predict

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd()
print(project_dir)

C:\Users\eikde\source\repos\UM_Crypto_Sentiment_Prediction


In [6]:
colors = {'red': '#ff207c', 'grey': '#42535b', 'blue': '#207cff', 'orange': '#ffa320', 'green': '#00ec8b'}
config_ticks = {'size': 14, 'color': colors['grey'], 'labelcolor': colors['grey']}
config_title = {'size': 18, 'color': colors['grey'], 'ha': 'left', 'va': 'baseline'}

In [7]:
#plot chart function
def get_charts(data, title):
    plt.rc('figure', figsize=(15, 10))
    fig, axes = plt.subplots(2, 1, 
                gridspec_kw={'height_ratios': [3, 1]})
    fig.tight_layout(pad=3)
    fig.suptitle(title, fontsize=16)
    
    date = data['Date']
    close = data['Close']
    vol = data['Volume']
    
    plot_price = axes[0]
    plot_price.plot(date, close, color=colors['blue'], 
    linewidth=2, label='Price')
    plot_price.set_ylabel('Price (in USD)', fontsize=14)
    plot_price.set_xlabel('Date', fontsize=14)
    
    plot_vol = axes[1]
    plot_vol.bar(date, vol, width=15, color='darkgrey')
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Volume (in millions)', fontsize=14)

In [8]:
#to check whether torch is available. Torch library is important for NLP and BERT.
torch.cuda.is_available()

True

# Preprocessing Text

Additional filter if needed, can be added on into preprocess_word function. This will help us to reuse this function to remove unnessary text. 

In [13]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#a function that to help preprocessing the message to a proper text for analysis
def preprocess_word(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    # Lowercase the message
    try:
        text = str(message).lower()
    except:
        print(text)
    
    
    # Replace % to percentage only fulfill for number
    try:
        replace_percent = re.findall('(\d+(\.\d+)?%)', text)
        for i in range(len(replace_percent)):
            item = re.sub('%', 'percent', replace_percent[i][0])
            percent = item.replace("percent","%")
            item = re.sub('%', ' percent', replace_percent[i][0])
            text = text.replace(percent, item)
    except:
        print(text)
        
    # Replace URLs with a space in the message
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    text = re.sub(r'http\S+', '', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    
    # Replace usernames with a space. The usernames are any word that starts with @.
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub('[^a-z0-9.0-9A-Z.]', ' ', text)
    
    # Remove stop words
    word_tokens = word_tokenize(remove_punctuation(text))
    
    # POS
    pos = nltk.pos_tag(word_tokens)
    
    lemmatizer=WordNetLemmatizer()
    
    text = ""
    for i in pos: 
        if i[1] == "IN" or i[1] =="PRP" or i[1] == "PRP$" or i[1] == "CC":
            text = text + ""
        else:
            #lemmatization
            text = text + lemmatizer.lemmatize(i[0]) + " "
        
    word_tokens = word_tokenize(text)

    
    filtered = []
    filtered = [w for w in word_tokens if not w in stop]
    filtered = TreebankWordDetokenizer().detokenize(filtered)
    
    return filtered

In [14]:
# Check whether the function is working. 
test_message = 'The news rapidly infected financial markets.'
print(preprocess_word(test_message))


news rapidly infected financial market


With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [13]:
#load back the pre-train model
cl_path = project_dir/'Models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [14]:
#Test statement
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [21]:
result = predict(preprocess_word(text),model, use_gpu=True, gpu_name='cuda:0',batch_size=100)

09/16/2022 19:34:50 - INFO - root -   Using device: cuda:0 
09/16/2022 19:34:50 - INFO - finbert.utils -   *** Example ***
09/16/2022 19:34:50 - INFO - finbert.utils -   guid: 0
09/16/2022 19:34:50 - INFO - finbert.utils -   tokens: [CLS] later day apple said rev ##ising earnings expectations fourth quarter 2018 largely lower sales signs economic weakness china . [SEP]
09/16/2022 19:34:50 - INFO - finbert.utils -   input_ids: 101 2101 2154 6207 2056 7065 9355 16565 10908 2959 4284 2760 4321 2896 4341 5751 3171 11251 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:34:50 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:34:50 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [22]:
result

,sentence,logit,prediction,sentiment_score
0,later day apple said revising earnings expecta...,"[0.0725401, 0.8918739, 0.035586055]",negative,-0.819334
1,news rapidly infected financial markets .,"[0.014923796, 0.91941434, 0.0656618]",negative,-0.904491
2,apple share price fell around 7 percent hours ...,"[0.005609312, 0.97903043, 0.015360256]",negative,-0.973421
3,dollar fell 3.7 percent yen matter minutes ann...,"[0.08141728, 0.8839122, 0.03467048]",negative,-0.802495
4,asian stockmarkets closed january 3rd european...,"[0.005998647, 0.9801152, 0.0138861425]",negative,-0.974117
5,yields government bonds fell investors fled tr...,"[0.018341133, 0.9392091, 0.042449817]",negative,-0.920868


In [23]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.90.


In [24]:
text = "STEPN is cutting its services for players in mainland China, and the move has had a major\
        impact on the app’s tokens. The move-to-earn lifestyle app announced Thursday that it would\
        cut access to users playing from mainland China to abide by local regulations."
result = predict(preprocess_word(text),model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
print(preprocess_word(text))


09/16/2022 19:34:57 - INFO - root -   Using device: cuda:0 
09/16/2022 19:34:57 - INFO - finbert.utils -   *** Example ***
09/16/2022 19:34:57 - INFO - finbert.utils -   guid: 0
09/16/2022 19:34:57 - INFO - finbert.utils -   tokens: [CLS] step ##n cutting services players mainland china move major impact app token ##s . [SEP]
09/16/2022 19:34:57 - INFO - finbert.utils -   input_ids: 101 3357 2078 6276 2578 2867 8240 2859 2693 2350 4254 10439 19204 2015 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:34:57 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:34:57 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:34:57 - INFO - finbert.utils -   label: No

stepn cutting services players mainland china move major impact app tokens . move earn lifestyle app announced thursday would cut access users playing mainland china abide local regulations.stepn cutting service player mainland china move ha major impact app token move earn lifestyle app announced thursday would cut access user playing mainland china abide local regulation


In [25]:
result


,sentence,logit,prediction,sentiment_score
0,stepn cutting services players mainland china ...,"[0.12552801, 0.5697212, 0.30475074]",negative,-0.444193
1,move earn lifestyle app announced thursday wou...,"[0.008645657, 0.9399462, 0.05140816]",negative,-0.931301


In [26]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.69.


In [27]:
text = "Good News For Markets As Fed’s Preferred Inflation Metric May Have Peaked. \
        Today’s PCE inflation number was relatively good news for markets suggesting inflation \
        could trend lower over the summer. The annual price change for April 2022 was 6.3%. \
        That’s down from 6.6% in March, as prices for goods rose at a slower pace than previously \
        and price increases for services rose at a broadly similar rate to recent months.\
        Stripping out food and energy, annual inflation fell back to 4.9%, a rate of growth\
        we last saw in December 2021. It’s too early to be sure, but inflation may be trending lower.\
        Of course, inflation is still well above the Federal Reserve’s 2% target,\
        but the direction is potentially a positive."
result = predict(preprocess_word(text),model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
print(preprocess_word(text))

09/16/2022 19:35:14 - INFO - root -   Using device: cuda:0 
09/16/2022 19:35:14 - INFO - finbert.utils -   *** Example ***
09/16/2022 19:35:14 - INFO - finbert.utils -   guid: 0
09/16/2022 19:35:14 - INFO - finbert.utils -   tokens: [CLS] good news markets fed preferred inflation metric may peaked . [SEP]
09/16/2022 19:35:14 - INFO - finbert.utils -   input_ids: 101 2204 2739 6089 7349 6871 14200 12046 2089 6601 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:35:14 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:35:14 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/16/2022 19:35:14 - INFO - finbert.utils -   label: None (id = 9090)
09/16/2022 19:35:1

good news markets fed preferred inflation metric may peaked . today pce inflation number relatively good news markets suggesting inflation could trend lower summer . annual price change april 2022 6.3 percent . 6.6 percent march prices goods rose slower pace previously price increases services rose broadly similar rate recent months . stripping food energy annual inflation fell back 4.9 percent rate growth last saw december 2021. early sure inflation may trending lower . course inflation still well federal reserve 2 percent target direction potentially positive.good news market fed preferred inflation metric may peaked today pce inflation number wa relatively good news market suggesting inflation could trend lower summer annual price change april 2022 wa 63 percent 66 percent march price good rose slower pace previously price increase service rose broadly similar rate recent month stripping food energy annual inflation fell back 49 percent rate growth last saw december 2021 early sure 

In [28]:
result

,sentence,logit,prediction,sentiment_score
0,good news markets fed preferred inflation metr...,"[0.3688417, 0.58910173, 0.042056616]",negative,-0.22026
1,today pce inflation number relatively good new...,"[0.9143363, 0.052312538, 0.033351112]",positive,0.862024
2,annual price change april 2022 6.3 percent .,"[0.036044456, 0.030975975, 0.9329796]",neutral,0.005068
3,6.6 percent march prices goods rose slower pac...,"[0.93278074, 0.027632844, 0.039586492]",positive,0.905148
4,stripping food energy annual inflation fell ba...,"[0.0753097, 0.89732313, 0.027367275]",negative,-0.822013
5,course inflation still well federal reserve 2 ...,"[0.27281946, 0.53881997, 0.18836065]",negative,-0.266001


In [29]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is 0.08.


# Generate Forbes finance news sentiment

In [30]:
#read excel file
news = pd.read_excel(r"Data/Forbes/Forbes_News_Jun2022.xlsx")
#news = news.drop(['Column1','sentiment_score'], axis=1)
news.rename(columns ={"label": "category"}, inplace=True)

In [31]:
start = dt.datetime(2021,10,1)
end = dt.datetime(2022,6,30)
def filter_news_by_date(start, end, data):
    data.date = pd.to_datetime(data.date)
    data = data[(data['date'] >= start) & (data['date'] <= end)]
    data = data.set_index('date') 
    data = data.sort_index()
    data = data.reset_index()
    #data = data.set_index('date')
    return data

In [32]:
news['category'] = np.where(news.category == 'Metaverse', 'Cryptocurrencies', news.category)
news['content'] = news["header"] + ". " + news["desc"]
news_target_date = filter_news_by_date(start, end, news)
news_target_date

,date,Column1,category,header,desc,author,content
0,2021-10-06,7330,Finance,India’s Wealth Creation: Rapid Recovery Puts E...,GDP grew at a record pace in the fiscal first ...,Rainer Michael Preiss,India’s Wealth Creation: Rapid Recovery Puts E...
1,2021-10-06,7342,Stock Market,McConnell Offers Democrats An Emergency Debt L...,"For weeks, the top Senate Republican has firml...",Jonathan Ponciano,McConnell Offers Democrats An Emergency Debt L...
2,2021-10-06,7341,Finance,How Making Public Long-Term Care Insurance (So...,When Washington State legislators approved a p...,Howard Gleckman,How Making Public Long-Term Care Insurance (So...
3,2021-10-06,7340,Stock Market,Blackstone Mortgage Trust Passes Through 8% Yi...,"In trading on Wednesday, shares of Blackstone ...",Dividend Channel,Blackstone Mortgage Trust Passes Through 8% Yi...
4,2021-10-06,7339,Finance,Why Traders Should Dig For Goldman Sachs Stock...,The shares of Goldman Sachs (GS) are up 0.4% a...,Schaeffer's Investment Research,Why Traders Should Dig For Goldman Sachs Stock...
...,...,...,...,...,...,...,...
9117,2022-06-30,7392,Stock Market,3 Key Student Loan Forgiveness Opportunities M...,Several new student loan forgiveness initiativ...,Adam S. Minsky,3 Key Student Loan Forgiveness Opportunities M...
9118,2022-06-30,7391,Cryptocurrencies,Supplementary Review: Progress Software (PRGS)...,Fiscal 2022 Q2 adjusted revenue (includes acqu...,Taesik Yoon,Supplementary Review: Progress Software (PRGS)...
9119,2022-06-30,7389,Stock Market,"Portfolio Change: One Stock To Buy, June 29",Investors’ fears that consumer demand will slo...,Taesik Yoon,"Portfolio Change: One Stock To Buy, June 29. I..."
9120,2022-06-30,7384,Finance,"Private Equity, Crypto Allowed In 401ks—Lotter...",Now that 401(k) investors are now free to jeop...,Edward Siedle,"Private Equity, Crypto Allowed In 401ks—Lotter..."


In [ ]:
finance_news = news_target_date.drop(['Column1','author'], axis=1)

sentiment_result = pd.DataFrame()
fieldnames = ['date','category','content', 'sentiment_score']

for index, row in finance_news.iterrows():
    date = row['date']
    content = preprocess_word(row['header']) + ' ' + preprocess_word(row['desc'])
    category = row['category']
    
    predict_content = predict(content, model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
    sentiment_score = predict_content.sentiment_score.mean()
    
    value = [(date, category, content, sentiment_score)]
    
    record = pd.DataFrame(value, columns=fieldnames)
    sentiment_result = pd.concat([sentiment_result, record], ignore_index=True, axis=0)

sentiment_result
sentiment_result.to_csv(r"Data/text_data/{0}_sentiment.csv".format('Forbes_News'), sep="\t")

09/16/2022 19:35:37 - INFO - root -   Using device: cuda:0 
09/16/2022 19:35:37 - INFO - finbert.utils -   *** Example ***
09/16/2022 19:35:37 - INFO - finbert.utils -   guid: 0
09/16/2022 19:35:37 - INFO - finbert.utils -   tokens: [CLS] india wealth creation rapid recovery puts economy track growth ##ind ##ia wealth creation rapid recovery put economy track growth gdp grew record pace fiscal first quarter though consumer demand private investment expected remain subdued due pan ##de ##mic . gdp grew record pace fiscal first quarter consumer demand private investment expected remain subdued due pan ##de ##mic [SEP]
09/16/2022 19:35:37 - INFO - finbert.utils -   input_ids: 101 2634 7177 4325 5915 7233 8509 4610 2650 3930 22254 2401 7177 4325 5915 7233 2404 4610 2650 3930 14230 3473 2501 6393 10807 2034 4284 2295 7325 5157 2797 5211 3517 3961 20442 2349 6090 3207 7712 1012 14230 3473 2501 6393 10807 2034 4284 7325 5157 2797 5211 3517 3961 20442 2349 6090 3207 7712 102 0 0 0 0 0
09/16/20

# Generate CoinDesk news sentiment

In [16]:
#read excel file
news = pd.read_excel(r"Data/CoinDesk/coindesk_news.xlsx")
#news = news.drop(['Column1','sentiment_score'], axis=1)
#news.rename(columns ={"label": "category"}, inplace=True)

In [19]:
news_target_date = filter_news_by_date(start, end, news)
news_target_date

,date,index,header,desc
0,2021-10-29,1121,Curve Finance’s CRV Quietly Becomes a Top-Perf...,CRV was up 1.87% in the past 24 hours and near...
1,2021-10-29,1118,"Play-to-Earn Squid Token Rockets 35,000% in 3 ...",The token buys entry to an online game inspire...
2,2021-10-29,1120,Bitcoin Back Over $60K as El Salvador Buys 420...,"After a swoon in recent days, the largest cryp..."
3,2021-10-29,1119,Market Wrap: Bitcoin Heads to $61K Ahead of Op...,Analysts expect short-term choppiness ahead of...
4,2021-10-30,1113,Ether alcanza un récord de $4400 mientras shib...,La quema de monedas se refiere al proceso de r...
...,...,...,...,...
1117,2022-06-30,4,Genesis Faces ‘Hundreds of Millions’ in Losses...,The DCG-owned trading colossus is said to have...
1118,2022-06-30,3,Messari Research: DCG’s Barry Silbert Wins Fro...,Messari’s Ryan Selkis says Grayscale's product...
1119,2022-06-30,2,First Mover Asia: How Traders Are Shorting Tet...,Hedge funds are increasingly betting against U...
1120,2022-06-30,1,Bitcoin Drops to Nearly $19K as Fed Renews Inf...,Central bank leaders warned Wednesday that inf...


In [ ]:
news_target_date['content'] = news_target_date["header"] + ". " + news_target_date["desc"]
coindesk_news = news_target_date.drop(['index','header','desc'], axis=1)

sentiment_result = pd.DataFrame()
fieldnames = ['date','content', 'sentiment_score']

for index, row in coindesk_news.iterrows():
    date = row['date']
    content = preprocess_word(row['content'])
    
    predict_content = predict(content, model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
    sentiment_score = predict_content.sentiment_score.mean()
    
    value = [(date, content, sentiment_score)]
    
    record = pd.DataFrame(value, columns=fieldnames)
    sentiment_result = pd.concat([sentiment_result, record], ignore_index=True, axis=0)

sentiment_result
sentiment_result.to_csv(r"Data/text_data/{0}_sentiment.csv".format('CoinDesk_News'), sep="\t")

07/11/2022 18:49:43 - INFO - root -   Using device: cuda:0 
07/11/2022 18:49:43 - INFO - finbert.utils -   *** Example ***
07/11/2022 18:49:43 - INFO - finbert.utils -   guid: 0
07/11/2022 18:49:43 - INFO - finbert.utils -   tokens: [CLS] curve finance cr ##v quietly becomes top performing def ##i token despite me ##me token cr ##az ##e . [SEP]
07/11/2022 18:49:43 - INFO - finbert.utils -   input_ids: 101 7774 5446 13675 2615 5168 4150 2327 4488 13366 2072 19204 2750 2033 4168 19204 13675 10936 2063 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 18:49:43 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 18:49:43 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 18:49:4

# Generate community discussion sentiment

In [42]:
def generate_sentiment(data, symbol):
    
    data["Process Content"] = data["Content"].apply(lambda x: preprocess_word(x))
    data["Date"] = data["Date"].apply(lambda x: pd.to_datetime(x, unit="ns", utc=True).floor('D').date())
    data = data.drop(['AuthorID','Author','Content', 'Attachments','Reactions'], axis=1)
    data = data.reset_index()
    sentiment_result = pd.DataFrame()
    fieldnames = ['symbol','date', 'content', 'sentiment_score']
    for index, row in data.iterrows():
        percent_complete(int(index), int(data.shape[0]),title="{0}: Predicting text sentiment analysis".format(symbol))
        symbol = symbol
        date = row['Date']
        content = row['Process Content']
        predict_content = predict(row['Process Content'], model, use_gpu=True, gpu_name='cuda:0',batch_size=100)
        sentiment_score = predict_content.sentiment_score.mean()
    
        value = [(symbol, date, content, sentiment_score)]
    
        record = pd.DataFrame(value, columns=fieldnames)
        sentiment_result = pd.concat([sentiment_result, record], ignore_index=True, axis=0)


    sentiment_result
    sentiment_result.to_csv(r"Data/text_data/{0}_sentiment.csv".format(symbol), sep="\t")

In [43]:
def percent_complete(step, total_steps, bar_width=60, title="", print_perc=True):
    import sys

    # UTF-8 left blocks: 1, 1/8, 1/4, 3/8, 1/2, 5/8, 3/4, 7/8
    utf_8s = ["█", "▏", "▎", "▍", "▌", "▋", "▊", "█"]
    perc = 100 * float(step) / float(total_steps)
    max_ticks = bar_width * 8
    num_ticks = int(round(perc / 100 * max_ticks))
    full_ticks = num_ticks / 8      # Number of full blocks
    part_ticks = num_ticks % 8      # Size of partial block (array index)
    
    disp = bar = ""                 # Blank out variables
    bar += utf_8s[0] * int(full_ticks)   # Add full blocks into Progress Bar
    
    # If part_ticks is zero, then no partial block, else append part char
    if part_ticks > 0:
        bar += utf_8s[part_ticks]
    
    # Pad Progress Bar with fill character
    bar += "▒" * int((max_ticks/8 - float(num_ticks)/8.0))
    
    if len(title) > 0:
        disp = title + ": "         # Optional title to progress display
    
    # Print progress bar in green: https://stackoverflow.com/a/21786287/6929343
    disp += "\x1b[0;32m"            # Color Green
    disp += bar                     # Progress bar to progress display
    disp += "\x1b[0m"               # Color Reset
    if print_perc:
        # If requested, append percentage complete to progress display
        if perc > 100.0:
            perc = 100.0            # Fix "100.04 %" rounding error
        disp += " {:6.2f}".format(perc) + " %"
    
    # Output to terminal repetitively over the same line using '\r'.
    sys.stdout.write("\r" + disp)
    sys.stdout.flush()

# Process Discord Data to Generate Sentiment Score

In [ ]:
MANA_General = pd.read_csv(r"Data/discord/Decentraland_General_2021-10-01_2022-06-30.csv")
generate_sentiment(MANA_General,'MANA_General')

In [ ]:
BTC_General = pd.read_csv(r"Data/discord/Bitcoin_General_2021-10-01_2022-06-30.csv")
generate_sentiment(BTC_General,'BTC_General')

In [ ]:
ETH_General = pd.read_csv(r"Data/discord/Ethereum_General_2021-10-01_2022-06-30.csv")
generate_sentiment(ETH_General,'ETH_General')

In [ ]:
BNB_General = pd.read_csv(r"Data/discord/Binance_General_2021-10-01_2022-06-30.csv")
generate_sentiment(BNB_General,'BNB_General')

In [ ]:
ENJ_General = pd.read_csv(r"Data/discord/Enjin_General_2021-10-01_2022-06-30.csv")
generate_sentiment(ENJ_General,'ENJ_General')

In [ ]:
RACA_General = pd.read_csv(r"Data/discord/RadioCaca_General2021-10-01_2022-06-30.csv")
generate_sentiment(RACA_General,'RACA_General')